In [ ]:
import cvxpy as cp
import numpy as np

class TrustRegionDescent:
    def __init__(self, delta, eta1=0.1, eta2=0.75, gamma1=0.5, gamma2=2.0):
        """
        Trust region descent method
        
        Parameters:
        - delta: Initial trust region radius
        - eta1: Improvement ratio threshold for shrinking trust region (default: 0.1)
        - eta2: Improvement ratio threshold for expanding trust region (default: 0.75)
        - gamma1: Shrink factor for trust region (default: 0.5)
        - gamma2: Expand factor for trust region (default: 2.0)
        """
        self.delta = delta
        self.eta1 = eta1
        self.eta2 = eta2
        self.gamma1 = gamma1
        self.gamma2 = gamma2
    
    def step(self, f, grad_f, H, x):
        """
        Perform one trust region step
        
        Parameters:
        - f: Objective function
        - ∇f: Gradient function
        - H: Hessian function
        - x: Current point
        
        Returns:
        - Updated point x
        """
        x_new, y_new = self.solve_trust_region_subproblem(grad_f, H, x, self.delta)
        
        # Compute improvement ratio
        eta = (f(x) - f(x_new)) / (f(x) - y_new) if (f(x) - y_new) != 0 else 0
        
        # Update trust region radius
        if eta < self.eta1:
            self.delta *= self.gamma1  # Shrink trust region
        elif eta > self.eta2:
            self.delta *= self.gamma2  # Expand trust region
        
        # Accept new point if improvement was sufficient
        return x_new if eta > self.eta1 else x
    
    def solve_trust_region_subproblem(self, grad_f, H, x0, delta):
        """
        Solve the trust region subproblem using quadratic approximation
        
        Parameters:
        - ∇f: Gradient function
        - H: Hessian function
        - x0: Current point
        - delta: Trust region radius
        
        Returns:
        - Tuple of (optimal point, optimal value)
        """
        n = len(x0)
        x = cp.Variable(n)
        
        # Quadratic approximation: f(x0) + ∇f(x0)ᵀ(x-x0) + ½(x-x0)ᵀH(x0)(x-x0)
        grad = grad_f(x0)
        hess = H(x0)
        quadratic_term = 0.5 * cp.quad_form(x - x0, hess)
        linear_term = grad @ (x - x0)
        
        objective = cp.Minimize(linear_term + quadratic_term)
        constraints = [cp.norm(x - x0) <= delta]
        problem = cp.Problem(objective, constraints)
        
        problem.solve(solver=cp.ECOS)
        
        return (x.value, problem.value)
    
# Example functions
def f(x): return x[0]**2 + x[1]**2  # Objective
def grad_f(x): return np.array([2*x[0], 2*x[1]])  # Gradient
def H(x): return np.array([[2, 0], [0, 2]])  # Hessian

# Initialize
tr = TrustRegionDescent(delta=1.0)
x_current = np.array([3.0, 4.0])

# Perform optimization steps
for _ in range(100):
    x_current = tr.step(f, grad_f, H, x_current)
    if np.linalg.norm(grad_f(x_current)) < 1e-6:
        break


[2.4 3.2]
[1.8 2.4]
[1.2 1.6]
[0.6 0.8]
[4.29163318e-06 5.72217871e-06]
[3.11418651e-10 4.15224616e-10]
